# OVOS Intent Classification — Comprehensive Multilingual Benchmark
## Training, Evaluation & Detailed Reporting Across All Languages

This notebook is a **full benchmark suite** for the OVOS intent-classification
dataset.  For every language in the dataset it:

1. **Streams & merges** all upstream splits, filters per language, and creates
   a balanced stratified train / val / test split.
2. **Trains** each applicable model (language-specific + multilingual baselines)
   using `model2vec` `StaticModelForClassification`.
3. **Evaluates** on the held-out test set and collects classification reports.
4. **Generates rich reports** including:
   - Per-model accuracy & macro-F1 tables
   - Confusion matrix heatmaps (top-N labels for readability)
   - Per-class F1 bar charts
   - Cross-language comparison radar / bar charts
   - Model size vs accuracy scatter plots

### Model sources

| Collection | URL |
|---|---|
| **minishlab / POTION** | [huggingface.co/collections/minishlab/potion](https://huggingface.co/collections/minishlab/potion) |
| **Jarbas / model2vec distilled** | [huggingface.co/collections/Jarbas/model2vec-distilled-models](https://huggingface.co/collections/Jarbas/model2vec-distilled-models) |

**Dataset:** [`OpenVoiceOS/ovos-intents-train-latest`](https://huggingface.co/datasets/OpenVoiceOS/ovos-intents-train-latest)

> **Credits:** Funded through the [NGI0 Commons Fund](https://nlnet.nl/commonsfund)
> via [NLnet](https://nlnet.nl), with support from the European Commission's
> Next Generation Internet programme (grant No 101135429).

---
## 0. Installs & Imports

In [5]:
import gc
import os
import re
import json
import random
import datetime
import warnings
import unicodedata
from pathlib import Path
from collections import Counter, defaultdict

import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from model2vec.train import StaticModelForClassification

import matplotlib
matplotlib.use("Agg")  # non-interactive backend for headless envs
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

try:
    import seaborn as sns
    HAS_SEABORN = True
except ImportError:
    HAS_SEABORN = False
    print("seaborn not found — falling back to matplotlib for heatmaps")

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    f1_score,
    accuracy_score,
)

warnings.filterwarnings("ignore", category=FutureWarning)
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 150,
    "savefig.bbox": "tight",
    "font.size": 9,
})
print("All imports OK")

All imports OK


In [4]:
#!pip install model2vec[train] datasets torch onnx onnxruntime numpy scikit-learn matplotlib seaborn

Using Python 3.11.14 environment at: /home/miro/PycharmProjects/wakeHuBert/.venv
Audited 9 packages in 13ms


---
## 1. Configuration

### 1a — Dataset & split parameters

In [6]:
DATASET_NAME  = os.environ.get("DATASET_NAME", "OpenVoiceOS/ovos-intents-train-latest")
SENTENCE_COL  = os.environ.get("SENTENCE_COL", "sentence")
LABEL_COL     = os.environ.get("LABEL_COL", "label")
LANG_COL      = os.environ.get("LANG_COL", "lang")
TEST_RATIO    = float(os.environ.get("TEST_RATIO", "0.15"))
VAL_RATIO     = float(os.environ.get("VAL_RATIO", "0.15"))
SEED          = int(os.environ.get("SEED", "42"))
MAX_EPOCHS    = int(os.environ.get("MAX_EPOCHS", "25"))
REPORT_DIR    = Path(os.environ.get("REPORT_DIR", "benchmark_reports"))
REPORT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Dataset:      {DATASET_NAME}")
print(f"Columns:      sentence={SENTENCE_COL!r}  label={LABEL_COL!r}  lang={LANG_COL!r}")
print(f"Split ratios: test={TEST_RATIO}  val={VAL_RATIO}")
print(f"Seed:         {SEED}")
print(f"Max epochs:   {MAX_EPOCHS}")
print(f"Reports dir:  {REPORT_DIR}")

Dataset:      OpenVoiceOS/ovos-intents-train-latest
Columns:      sentence='sentence'  label='label'  lang='lang'
Split ratios: test=0.15  val=0.15
Seed:         42
Max epochs:   25
Reports dir:  benchmark_reports


### 1b — Model registry

For each language we define which models to benchmark.  Every language gets:
- A set of **multilingual** models (shared across all languages)
- **Language-specific** models distilled from monolingual sentence transformers

English-only potion models serve as baselines to show how monolingual EN
embeddings degrade on non-English data.

> **Note:** If a model fails to download or train (e.g. gated model), it is
> skipped gracefully and reported in the summary.

In [7]:
# ── Multilingual models (tested on every language) ─────────────
MULTILINGUAL_MODELS = [
    # minishlab POTION
    "minishlab/potion-multilingual-128M",
]

# ── Language-specific models ───────────────────────────────────
LANG_SPECIFIC_MODELS = {
    "en": [
        "minishlab/potion-base-2M",
        "minishlab/potion-base-4M",
        "minishlab/potion-base-8M",
        "minishlab/potion-base-32M",
       # "minishlab/potion-retrieval-32M",
    ],
    "pt": [
        "Jarbas/m2v-256-bert-base-portuguese-cased",
        "Jarbas/m2v-256-bert-large-portuguese-cased",
        "Jarbas/m2v-256-albertina-100m-portuguese-ptpt-encoder",
        "Jarbas/m2v-256-albertina-100m-portuguese-ptbr-encoder",
        "Jarbas/m2v-256-serafim-100m-portuguese-pt-sentence-encoder",
        "Jarbas/m2v-256-serafim-335m-portuguese-pt-sentence-encoder",
    ],
    "ca": [
        "Jarbas/m2v-256-roberta-large-ca-v2",
        "Jarbas/m2v-256-roberta-large-ca-v2-massive",
        "Jarbas/m2v-256-roberta-base-ca-v2-massive",
        "Jarbas/m2v-256-distilroberta-base-ca-v2",
        "Jarbas/m2v-256-roberta-large-ca-paraphrase",
        "Jarbas/m2v-256-roberta-base-ca-cased-sts",
    ],
    "gl": [
        "Jarbas/m2v-256-bertinho-gl-base-cased",
        "Jarbas/m2v-256-bertinho-gl-small-cased",
    ],
    "es": [
        "Jarbas/m2v-256-roberta-large-bne",
        "Jarbas/m2v-256-xlm-roberta-large-finetuned-conll02-spanish",
    ],
    "de": [
        "Jarbas/m2v-256-xlm-roberta-large-finetuned-conll03-german",
    ],
    "nl": [
        "Jarbas/m2v-256-xlm-roberta-large-finetuned-conll02-dutch",
    ],
    "da": [
        "Jarbas/m2v-256-danish-bert-botxo",
    ],
    "eu": [
        "Jarbas/m2v-256-BERnaT-base",
        "Jarbas/m2v-256-BERnaT-base-NERC",
        "Jarbas/m2v-256-BERnaT-large-NERC",
        "Jarbas/m2v-256-EriBERTa-base",
    ],
    "it": [],  # No language-specific models — multilingual only
    "fr": [],  # No language-specific models — multilingual only
}

def get_models_for_lang(lang: str) -> list[str]:
    """Return multilingual + language-specific models for a language."""
    specific = LANG_SPECIFIC_MODELS.get(lang, [])
    return MULTILINGUAL_MODELS + specific

# Summary
total_runs = sum(len(get_models_for_lang(l)) for l in LANG_SPECIFIC_MODELS)
print(f"Languages:          {len(LANG_SPECIFIC_MODELS)}")
print(f"Multilingual models: {len(MULTILINGUAL_MODELS)}")
print(f"Total training runs: {total_runs}")
for lang in sorted(LANG_SPECIFIC_MODELS):
    models = get_models_for_lang(lang)
    n_specific = len(LANG_SPECIFIC_MODELS.get(lang, []))
    print(f"  {lang}: {len(models)} models ({n_specific} lang-specific)")

Languages:          11
Multilingual models: 1
Total training runs: 38
  ca: 7 models (6 lang-specific)
  da: 2 models (1 lang-specific)
  de: 2 models (1 lang-specific)
  en: 5 models (4 lang-specific)
  es: 3 models (2 lang-specific)
  eu: 5 models (4 lang-specific)
  fr: 1 models (0 lang-specific)
  gl: 3 models (2 lang-specific)
  it: 1 models (0 lang-specific)
  nl: 2 models (1 lang-specific)
  pt: 7 models (6 lang-specific)


---
## 2. Stream, Merge & Filter Dataset

We stream all three upstream splits into a single pool, then split per language.
The upstream splits are **not balanced**, so we merge them and create our own
stratified split per language.

In [8]:
def stream_all_splits(dataset_name: str) -> list[dict]:
    """Stream all splits into a flat list of dicts."""
    all_rows = []
    for split_name in ("train", "validation", "test"):
        ds = load_dataset(dataset_name, split=split_name, streaming=True)
        for row in ds:
            all_rows.append({
                "sentence": row[SENTENCE_COL],
                "label":    row[LABEL_COL],
                "lang":     row[LANG_COL],
            })
    return all_rows


print("Streaming all splits …")
all_rows = stream_all_splits(DATASET_NAME)
print(f"Total rows: {len(all_rows)}")

# Group by language
rows_by_lang: dict[str, list[dict]] = defaultdict(list)
for r in all_rows:
    rows_by_lang[r["lang"]].append(r)

del all_rows
gc.collect()

print(f"\nPer-language counts:")
for lang in sorted(rows_by_lang):
    print(f"  {lang}: {len(rows_by_lang[lang]):>6} rows, "
          f"{len(set(r['label'] for r in rows_by_lang[lang]))} labels")

Streaming all splits …
Total rows: 122178

Per-language counts:
  ca:  43448 rows, 135 labels
  da:   6340 rows, 93 labels
  de:  11266 rows, 148 labels
  en:  12144 rows, 157 labels
  es:   5280 rows, 129 labels
  eu:   1687 rows, 90 labels
  fr:   4641 rows, 124 labels
  gl:  17542 rows, 136 labels
  it:   8414 rows, 136 labels
  nl:   5733 rows, 92 labels
  pt:   5683 rows, 130 labels


---
## 3. Balanced Stratified Split Utility

In [9]:
def stratified_split(
    rows: list[dict],
    test_ratio: float,
    val_ratio: float,
    seed: int,
) -> tuple[dict, dict, dict]:
    """Stratified split preserving label distribution."""
    rng = random.Random(seed)
    by_label: dict[str, list[dict]] = {}
    for r in rows:
        by_label.setdefault(r["label"], []).append(r)

    train_rows, val_rows, test_rows = [], [], []
    for label, items in by_label.items():
        rng.shuffle(items)
        n = len(items)
        if n < 3:
            train_rows.extend(items)
            continue
        n_test = max(1, int(round(n * test_ratio)))
        n_val  = max(1, int(round(n * val_ratio)))
        if n_test + n_val >= n:
            n_test = max(1, n // 3)
            n_val  = max(1, n // 3)
        test_rows.extend(items[:n_test])
        val_rows.extend(items[n_test : n_test + n_val])
        train_rows.extend(items[n_test + n_val :])

    rng.shuffle(train_rows)
    rng.shuffle(val_rows)
    rng.shuffle(test_rows)

    def to_dict(rows):
        return {
            "sentence": [r["sentence"] for r in rows],
            "label":    [r["label"]    for r in rows],
        }
    return to_dict(train_rows), to_dict(val_rows), to_dict(test_rows)

print("Splitting each language …")
splits_by_lang: dict[str, tuple[dict, dict, dict]] = {}
for lang in sorted(rows_by_lang):
    train, val, test = stratified_split(rows_by_lang[lang], TEST_RATIO, VAL_RATIO, SEED)
    splits_by_lang[lang] = (train, val, test)
    n_labels = len(set(train["label"]))
    print(f"  {lang}: train={len(train['sentence']):>5}  "
          f"val={len(val['sentence']):>5}  "
          f"test={len(test['sentence']):>5}  "
          f"labels={n_labels}")

# Free raw rows
del rows_by_lang
gc.collect()

Splitting each language …
  ca: train=30400  val= 6524  test= 6524  labels=135
  da: train= 4424  val=  958  test=  958  labels=93
  de: train= 7854  val= 1706  test= 1706  labels=148
  en: train= 8488  val= 1828  test= 1828  labels=157
  es: train= 3642  val=  819  test=  819  labels=129
  eu: train= 1161  val=  263  test=  263  labels=90
  fr: train= 3229  val=  706  test=  706  labels=124
  gl: train=12252  val= 2645  test= 2645  labels=136
  it: train= 5864  val= 1275  test= 1275  labels=136
  nl: train= 3991  val=  871  test=  871  labels=92
  pt: train= 3951  val=  866  test=  866  labels=130


0

---
## 4. Reporting Helpers

In [10]:
def short_name(model_name: str) -> str:
    """Abbreviate a HF model name for display."""
    return model_name.split("/")[-1]


def plot_confusion_matrix(
    y_true, y_pred, labels, title: str, save_path: Path, top_n: int = 30,
):
    """Plot a confusion matrix heatmap for the top-N most frequent labels."""
    # Select top-N labels by support in y_true
    label_counts = Counter(y_true)
    top_labels = [l for l, _ in label_counts.most_common(top_n)]
    # Filter to only samples with top labels
    mask = [(t in top_labels and p in top_labels) for t, p in zip(y_true, y_pred)]
    y_t = [t for t, m in zip(y_true, mask) if m]
    y_p = [p for p, m in zip(y_pred, mask) if m]
    if len(y_t) == 0:
        return

    # Reorder labels by frequency
    cm = confusion_matrix(y_t, y_p, labels=top_labels)

    fig, ax = plt.subplots(figsize=(max(10, top_n * 0.4), max(8, top_n * 0.35)))
    if HAS_SEABORN:
        sns.heatmap(
            cm, annot=False, fmt="d", cmap="Blues",
            xticklabels=top_labels, yticklabels=top_labels, ax=ax,
            linewidths=0.3, linecolor="white",
        )
    else:
        im = ax.imshow(cm, interpolation="nearest", cmap="Blues")
        fig.colorbar(im, ax=ax, shrink=0.7)
        ax.set_xticks(range(len(top_labels)))
        ax.set_yticks(range(len(top_labels)))
        ax.set_xticklabels(top_labels, rotation=90, fontsize=6)
        ax.set_yticklabels(top_labels, fontsize=6)

    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(title, fontsize=11)
    plt.tight_layout()
    fig.savefig(save_path)
    plt.close(fig)
    print(f"  Saved: {save_path}")


def plot_per_class_f1(
    y_true, y_pred, title: str, save_path: Path, top_n: int = 30,
):
    """Horizontal bar chart of per-class F1 for the top-N labels by support."""
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    # Filter to actual label keys (not 'accuracy', 'macro avg', etc.)
    avg_keys = {"accuracy", "macro avg", "weighted avg"}
    items = [(k, v["f1-score"], v["support"])
             for k, v in report.items() if k not in avg_keys]
    # Sort by support, take top_n
    items.sort(key=lambda x: -x[2])
    items = items[:top_n]
    # Sort by F1 for display
    items.sort(key=lambda x: x[1])

    labels_plot = [it[0][:40] for it in items]
    f1s = [it[1] for it in items]
    supports = [it[2] for it in items]

    fig, ax = plt.subplots(figsize=(8, max(5, len(items) * 0.22)))
    colors = plt.cm.RdYlGn(np.array(f1s))
    bars = ax.barh(labels_plot, f1s, color=colors, edgecolor="white", linewidth=0.5)

    # Add support count annotations
    for bar, s in zip(bars, supports):
        ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height() / 2,
                f"n={s}", va="center", fontsize=6, color="gray")

    ax.set_xlim(0, 1.15)
    ax.set_xlabel("F1-score")
    ax.set_title(title, fontsize=11)
    ax.axvline(x=0.5, color="red", linestyle="--", alpha=0.3, linewidth=0.8)
    ax.axvline(x=0.9, color="green", linestyle="--", alpha=0.3, linewidth=0.8)
    plt.tight_layout()
    fig.savefig(save_path)
    plt.close(fig)
    print(f"  Saved: {save_path}")


def plot_model_comparison(
    results_for_lang: dict, lang: str, save_path: Path,
):
    """Bar chart comparing all models for a given language."""
    names = []
    accs = []
    f1s = []
    for model_name, res in results_for_lang.items():
        names.append(short_name(model_name))
        accs.append(res["accuracy"])
        f1s.append(res["macro_f1"])

    # Sort by macro F1
    order = np.argsort(f1s)
    names = [names[i] for i in order]
    accs = [accs[i] for i in order]
    f1s = [f1s[i] for i in order]

    fig, ax = plt.subplots(figsize=(10, max(4, len(names) * 0.3)))
    y = np.arange(len(names))
    h = 0.35
    ax.barh(y - h/2, accs, h, label="Accuracy", color="#4C72B0", alpha=0.85)
    ax.barh(y + h/2, f1s, h, label="Macro F1", color="#DD8452", alpha=0.85)

    ax.set_yticks(y)
    ax.set_yticklabels(names, fontsize=7)
    ax.set_xlabel("Score")
    ax.set_xlim(0, 1.05)
    ax.set_title(f"Model Comparison — {lang.upper()}", fontsize=12)
    ax.legend(loc="lower right", fontsize=8)
    ax.axvline(x=0.9, color="green", linestyle="--", alpha=0.3, linewidth=0.8)
    plt.tight_layout()
    fig.savefig(save_path)
    plt.close(fig)
    print(f"  Saved: {save_path}")

print("Plotting helpers ready")

Plotting helpers ready


---
## 5. Main Benchmark Loop

This is the core of the notebook.  For each language we:
1. Retrieve the pre-computed balanced split.
2. Iterate over all applicable models (multilingual + lang-specific).
3. Train, evaluate, and store detailed results.
4. Generate per-model confusion matrix & F1 plots.
5. Generate a per-language model comparison chart.

Models that fail to load are skipped with a warning.

In [11]:
# ── Global results store ─────────────────────────────────────────
ALL_RESULTS: dict[str, dict[str, dict]] = {}   # lang -> model -> metrics
FAILED_MODELS: list[tuple[str, str, str]] = []  # (lang, model, error)

for lang in sorted(splits_by_lang):
    train_data, val_data, test_data = splits_by_lang[lang]
    models = get_models_for_lang(lang)
    lang_dir = REPORT_DIR / lang
    lang_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n{'#'*70}")
    print(f"#  Language: {lang.upper()}  ({len(train_data['sentence'])} train, "
          f"{len(test_data['sentence'])} test, {len(models)} models)")
    print(f"{'#'*70}")

    lang_results = {}

    for model_name in models:
        sname = short_name(model_name)
        print(f"\n  ── {model_name} ──")

        try:
            classifier = StaticModelForClassification.from_pretrained(
                model_name=model_name
            )
        except Exception as e:
            msg = str(e)[:120]
            print(f"  ⚠ SKIP (load failed): {msg}")
            FAILED_MODELS.append((lang, model_name, msg))
            continue

        try:
            classifier.fit(
                max_epochs=MAX_EPOCHS,
                X=train_data["sentence"],
                y=train_data["label"],
                X_val=val_data["sentence"],
                y_val=val_data["label"],
            )
        except Exception as e:
            msg = str(e)[:120]
            print(f"  ⚠ SKIP (train failed): {msg}")
            FAILED_MODELS.append((lang, model_name, msg))
            del classifier; gc.collect()
            continue

        # ── Evaluate ──
        try:
            y_pred = classifier.predict(test_data["sentence"])
        except Exception:
            # Fallback: batch predict
            y_pred = []
            for i in range(0, len(test_data["sentence"]), 256):
                y_pred.extend(classifier.predict(test_data["sentence"][i:i+256]))

        y_true = test_data["label"]
        acc = accuracy_score(y_true, y_pred)
        macro_f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
        weighted_f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)

        report_str = classification_report(y_true, y_pred, zero_division=0)
        report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

        print(f"  Accuracy: {acc:.4f}  |  Macro F1: {macro_f1:.4f}  |  Weighted F1: {weighted_f1:.4f}")

        # ── Store ──
        lang_results[model_name] = {
            "accuracy": acc,
            "macro_f1": macro_f1,
            "weighted_f1": weighted_f1,
            "report_str": report_str,
            "report_dict": report_dict,
            "y_true": y_true,
            "y_pred": y_pred,
        }

        # ── Per-model plots ──
        plot_confusion_matrix(
            y_true, y_pred,
            labels=sorted(set(y_true)),
            title=f"Confusion Matrix — {sname} [{lang}]",
            save_path=lang_dir / f"cm_{sname}.png",
            top_n=25,
        )
        plot_per_class_f1(
            y_true, y_pred,
            title=f"Per-Class F1 — {sname} [{lang}]",
            save_path=lang_dir / f"f1_{sname}.png",
            top_n=30,
        )

        # ── Cleanup ──
        del classifier
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # ── Per-language comparison plot ──
    if lang_results:
        plot_model_comparison(lang_results, lang, lang_dir / "model_comparison.png")

    ALL_RESULTS[lang] = lang_results

print(f"\n{'='*70}")
print(f"Benchmark complete!  {sum(len(v) for v in ALL_RESULTS.values())} successful runs, "
      f"{len(FAILED_MODELS)} failures")
print(f"{'='*70}")


######################################################################
#  Language: CA  (30400 train, 6524 test, 7 models)
######################################################################

  ── minishlab/potion-multilingual-128M ──


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
/home/miro/.jupyter_env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckp

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type                         ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model         │ StaticModelForClassification │  129 M │ train │     0 │
│ 1 │ loss_function │ CrossEntropyLoss             │      0 │ train │     0 │
└───┴───────────────┴──────────────────────────────┴────────┴───────┴───────┘

Trainable params: 128 M                                                                                            
Non-trainable params: 500 K                                                                                        
Total params: 129 M                                                                                                
Total estimated model params size (MB): 517                                                                        
Modules in train mode: 7                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

...

/home/miro/.jupyter_env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: 
The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

/home/miro/.jupyter_env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: 
The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

/home/miro/.jupyter_env/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:317: The number of 
training batches (31) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for 
log_every_n_steps if you want to see logs for the training epoch.

`Trainer.fit` stopped: `max_epochs=25` reached.


  Accuracy: 0.9457  |  Macro F1: 0.7415  |  Weighted F1: 0.9371
  Saved: benchmark_reports/pt/cm_m2v-256-serafim-335m-portuguese-pt-sentence-encoder.png
  Saved: benchmark_reports/pt/f1_m2v-256-serafim-335m-portuguese-pt-sentence-encoder.png
  Saved: benchmark_reports/pt/model_comparison.png

Benchmark complete!  38 successful runs, 0 failures


---
## 6. Results Tables

### 6a — Per-Language Leaderboard

In [12]:
for lang in sorted(ALL_RESULTS):
    results = ALL_RESULTS[lang]
    if not results:
        continue
    print(f"\n{'='*75}")
    print(f"  {lang.upper()} — {len(results)} models")
    print(f"{'='*75}")
    print(f"  {'Model':<48} {'Acc':>7} {'F1-mac':>8} {'F1-wgt':>8}")
    print(f"  {'-'*71}")

    # Sort by macro F1 descending
    ranked = sorted(results.items(), key=lambda x: -x[1]["macro_f1"])
    for i, (model_name, res) in enumerate(ranked):
        marker = "🏆" if i == 0 else "  "
        print(f"{marker} {short_name(model_name):<48} "
              f"{res['accuracy']:>7.4f} {res['macro_f1']:>8.4f} {res['weighted_f1']:>8.4f}")


  CA — 7 models
  Model                                                Acc   F1-mac   F1-wgt
  -----------------------------------------------------------------------
🏆 m2v-256-roberta-base-ca-v2-massive                0.9920   0.8506   0.9912
   m2v-256-roberta-large-ca-v2-massive               0.9919   0.8345   0.9909
   m2v-256-distilroberta-base-ca-v2                  0.9890   0.8118   0.9879
   m2v-256-roberta-large-ca-v2                       0.9896   0.8101   0.9884
   m2v-256-roberta-base-ca-cased-sts                 0.9903   0.8017   0.9892
   m2v-256-roberta-large-ca-paraphrase               0.9888   0.7974   0.9875
   potion-multilingual-128M                          0.9905   0.7809   0.9891

  DA — 2 models
  Model                                                Acc   F1-mac   F1-wgt
  -----------------------------------------------------------------------
🏆 potion-multilingual-128M                          0.9666   0.8723   0.9635
   m2v-256-danish-bert-botxo              

### 6b — Cross-Language Best-Model Table

In [13]:
print(f"{'Lang':<6} {'Best Model':<48} {'Acc':>7} {'F1-mac':>8}")
print("-" * 72)
for lang in sorted(ALL_RESULTS):
    results = ALL_RESULTS[lang]
    if not results:
        continue
    best_model = max(results, key=lambda m: results[m]["macro_f1"])
    res = results[best_model]
    print(f"{lang:<6} {short_name(best_model):<48} "
          f"{res['accuracy']:>7.4f} {res['macro_f1']:>8.4f}")

Lang   Best Model                                           Acc   F1-mac
------------------------------------------------------------------------
ca     m2v-256-roberta-base-ca-v2-massive                0.9920   0.8506
da     potion-multilingual-128M                          0.9666   0.8723
de     potion-multilingual-128M                          0.9654   0.8407
en     potion-multilingual-128M                          0.9683   0.8634
es     potion-multilingual-128M                          0.9341   0.7709
eu     m2v-256-BERnaT-base                               0.9392   0.8219
fr     potion-multilingual-128M                          0.9448   0.7728
gl     potion-multilingual-128M                          0.9890   0.8508
it     potion-multilingual-128M                          0.9647   0.7665
nl     potion-multilingual-128M                          0.9414   0.8051
pt     m2v-256-bert-base-portuguese-cased                0.9423   0.7642


### 6c — Multilingual Models Across Languages (Heatmap)

In [14]:
# Build a matrix: multilingual models × languages
multi_models_present = []
for m in MULTILINGUAL_MODELS:
    for lang in sorted(ALL_RESULTS):
        if m in ALL_RESULTS[lang]:
            if m not in multi_models_present:
                multi_models_present.append(m)
            break

langs_sorted = sorted(ALL_RESULTS.keys())

if multi_models_present and langs_sorted:
    matrix = np.zeros((len(multi_models_present), len(langs_sorted)))
    for i, m in enumerate(multi_models_present):
        for j, lang in enumerate(langs_sorted):
            if m in ALL_RESULTS.get(lang, {}):
                matrix[i, j] = ALL_RESULTS[lang][m]["macro_f1"]
            else:
                matrix[i, j] = np.nan

    fig, ax = plt.subplots(
        figsize=(max(8, len(langs_sorted) * 0.9),
                 max(6, len(multi_models_present) * 0.4))
    )
    model_labels = [short_name(m) for m in multi_models_present]

    if HAS_SEABORN:
        sns.heatmap(
            matrix, annot=True, fmt=".3f", cmap="YlOrRd",
            xticklabels=langs_sorted, yticklabels=model_labels,
            ax=ax, vmin=0.5, vmax=1.0,
            linewidths=0.5, linecolor="white",
        )
    else:
        im = ax.imshow(matrix, cmap="YlOrRd", vmin=0.5, vmax=1.0, aspect="auto")
        fig.colorbar(im, ax=ax, shrink=0.8)
        ax.set_xticks(range(len(langs_sorted)))
        ax.set_xticklabels(langs_sorted)
        ax.set_yticks(range(len(multi_models_present)))
        ax.set_yticklabels(model_labels, fontsize=7)
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if not np.isnan(matrix[i, j]):
                    ax.text(j, i, f"{matrix[i,j]:.3f}",
                            ha="center", va="center", fontsize=6)

    ax.set_title("Macro F1 — Multilingual Models × Languages", fontsize=12)
    ax.set_xlabel("Language")
    plt.tight_layout()
    save_p = REPORT_DIR / "multilingual_heatmap.png"
    fig.savefig(save_p)
    plt.close(fig)
    print(f"Saved: {save_p}")
    plt.figure(figsize=(1,1)); plt.close()  # reset
else:
    print("No multilingual results to plot.")

Saved: benchmark_reports/multilingual_heatmap.png


---
## 7. Cross-Language Analysis Plots

### 7a — Best Accuracy per Language

In [15]:
langs_sorted = sorted(ALL_RESULTS.keys())
best_accs = []
best_f1s = []
best_names = []
for lang in langs_sorted:
    results = ALL_RESULTS[lang]
    if not results:
        best_accs.append(0); best_f1s.append(0); best_names.append("N/A")
        continue
    best_m = max(results, key=lambda m: results[m]["macro_f1"])
    best_accs.append(results[best_m]["accuracy"])
    best_f1s.append(results[best_m]["macro_f1"])
    best_names.append(short_name(best_m))

fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(langs_sorted))
w = 0.35
bars1 = ax.bar(x - w/2, best_accs, w, label="Accuracy", color="#4C72B0", alpha=0.85)
bars2 = ax.bar(x + w/2, best_f1s, w, label="Macro F1", color="#DD8452", alpha=0.85)

ax.set_xticks(x)
ax.set_xticklabels([f"{l}\n({n[:20]})" for l, n in zip(langs_sorted, best_names)],
                    fontsize=7, rotation=0)
ax.set_ylabel("Score")
ax.set_ylim(0, 1.05)
ax.set_title("Best Model Performance per Language", fontsize=13)
ax.legend()
ax.axhline(y=0.9, color="green", linestyle="--", alpha=0.3)
plt.tight_layout()
fig.savefig(REPORT_DIR / "best_per_language.png")
plt.close(fig)
print(f"Saved: {REPORT_DIR / 'best_per_language.png'}")

Saved: benchmark_reports/best_per_language.png


### 7b — Language-Specific vs Multilingual-Only

In [16]:
# For each language, compare the best lang-specific model vs best multilingual
fig, ax = plt.subplots(figsize=(10, 5))

langs_with_specific = [l for l in langs_sorted if LANG_SPECIFIC_MODELS.get(l)]
x = np.arange(len(langs_with_specific))
multi_best = []
specific_best = []

for lang in langs_with_specific:
    results = ALL_RESULTS.get(lang, {})
    # Best multilingual
    multi_f1 = 0
    for m in MULTILINGUAL_MODELS:
        if m in results:
            multi_f1 = max(multi_f1, results[m]["macro_f1"])
    multi_best.append(multi_f1)
    # Best lang-specific
    spec_f1 = 0
    for m in LANG_SPECIFIC_MODELS.get(lang, []):
        if m in results:
            spec_f1 = max(spec_f1, results[m]["macro_f1"])
    specific_best.append(spec_f1)

w = 0.35
ax.bar(x - w/2, multi_best, w, label="Best Multilingual", color="#4C72B0")
ax.bar(x + w/2, specific_best, w, label="Best Lang-Specific", color="#C44E52")
ax.set_xticks(x)
ax.set_xticklabels(langs_with_specific, fontsize=9)
ax.set_ylabel("Macro F1")
ax.set_ylim(0, 1.05)
ax.set_title("Multilingual vs Language-Specific Models", fontsize=13)
ax.legend()
plt.tight_layout()
fig.savefig(REPORT_DIR / "multi_vs_specific.png")
plt.close(fig)
print(f"Saved: {REPORT_DIR / 'multi_vs_specific.png'}")

Saved: benchmark_reports/multi_vs_specific.png


### 7c — POTION Model Size vs Accuracy

In [17]:
# Compare potion models across languages
potion_models = {
    "minishlab/potion-base-2M": 2,
    "minishlab/potion-base-4M": 4,
    "minishlab/potion-base-8M": 8,
    "minishlab/potion-base-32M": 32,
    "minishlab/potion-multilingual-128M": 128,
}

fig, ax = plt.subplots(figsize=(10, 6))
cmap = plt.cm.tab10
for idx, lang in enumerate(langs_sorted):
    results = ALL_RESULTS.get(lang, {})
    sizes = []
    f1s = []
    for m, sz in potion_models.items():
        if m in results:
            sizes.append(sz)
            f1s.append(results[m]["macro_f1"])
    if sizes:
        ax.plot(sizes, f1s, "o-", label=lang, color=cmap(idx), markersize=5, linewidth=1.5)

ax.set_xscale("log")
ax.set_xticks(list(potion_models.values()))
ax.get_xaxis().set_major_formatter(mticker.ScalarFormatter())
ax.set_xlabel("Model Size (M params)")
ax.set_ylabel("Macro F1")
ax.set_ylim(0, 1.05)
ax.set_title("POTION: Model Size vs Performance by Language", fontsize=13)
ax.legend(fontsize=8, ncol=3, loc="lower right")
ax.grid(True, alpha=0.3)
plt.tight_layout()
fig.savefig(REPORT_DIR / "potion_size_vs_f1.png")
plt.close(fig)
print(f"Saved: {REPORT_DIR / 'potion_size_vs_f1.png'}")

Saved: benchmark_reports/potion_size_vs_f1.png


---
## 8. Detailed Classification Reports (Best Model per Language)

Print the full `sklearn` classification report for the best model in each
language.

In [18]:
for lang in sorted(ALL_RESULTS):
    results = ALL_RESULTS[lang]
    if not results:
        continue
    best_model = max(results, key=lambda m: results[m]["macro_f1"])
    res = results[best_model]

    print(f"\n{'='*75}")
    print(f"  {lang.upper()} — Best: {short_name(best_model)}")
    print(f"  Accuracy={res['accuracy']:.4f}  Macro-F1={res['macro_f1']:.4f}")
    print(f"{'='*75}")
    print(res["report_str"][:3000])
    if len(res["report_str"]) > 3000:
        print("  … (truncated)")


  CA — Best: m2v-256-roberta-base-ca-v2-massive
  Accuracy=0.9920  Macro-F1=0.8506
                                                                              precision    recall  f1-score   support

                                                   common_query:common_query       0.93      0.91      0.92       168
                                                                    ocp:play       0.97      0.97      0.97       275
                               ovos-skill-alerts.openvoiceos:AddListSubitems       1.00      1.00      1.00         2
                                  ovos-skill-alerts.openvoiceos:CalendarList       1.00      1.00      1.00         1
                                   ovos-skill-alerts.openvoiceos:CancelAlert       1.00      1.00      1.00         1
                                   ovos-skill-alerts.openvoiceos:CreateAlarm       1.00      1.00      1.00         1
                                   ovos-skill-alerts.openvoiceos:CreateEvent       1.00  

---
## 9. Failed Models

In [19]:
if FAILED_MODELS:
    print(f"\n{len(FAILED_MODELS)} model(s) failed to load or train:\n")
    for lang, model, err in FAILED_MODELS:
        print(f"  [{lang}] {model}")
        print(f"         {err}")
else:
    print("All models loaded and trained successfully!")

All models loaded and trained successfully!


---
## 10. Export Metadata

In [20]:
meta = {
    "task": "intent-classification-benchmark",
    "dataset": DATASET_NAME,
    "seed": SEED,
    "max_epochs": MAX_EPOCHS,
    "split_ratios": {
        "train": round(1 - TEST_RATIO - VAL_RATIO, 4),
        "validation": VAL_RATIO,
        "test": TEST_RATIO,
    },
    "languages": {},
    "failed": [
        {"lang": l, "model": m, "error": e} for l, m, e in FAILED_MODELS
    ],
    "exported": datetime.datetime.utcnow().isoformat() + "Z",
}

for lang in sorted(ALL_RESULTS):
    results = ALL_RESULTS[lang]
    train_d, val_d, test_d = splits_by_lang[lang]
    lang_meta = {
        "split_sizes": {
            "train": len(train_d["sentence"]),
            "validation": len(val_d["sentence"]),
            "test": len(test_d["sentence"]),
        },
        "num_labels": len(set(train_d["label"])),
        "models": {},
    }
    for model_name, res in results.items():
        lang_meta["models"][model_name] = {
            "accuracy": round(res["accuracy"], 5),
            "macro_f1": round(res["macro_f1"], 5),
            "weighted_f1": round(res["weighted_f1"], 5),
        }
    meta["languages"][lang] = lang_meta

meta_path = REPORT_DIR / "benchmark_metadata.json"
meta_path.write_text(json.dumps(meta, indent=2))
print(f"Metadata saved to {meta_path}")

Metadata saved to benchmark_reports/benchmark_metadata.json


### Report files generated

In [21]:
import glob
report_files = sorted(glob.glob(str(REPORT_DIR / "**" / "*"), recursive=True))
print(f"\n{len(report_files)} files in {REPORT_DIR}/:")
for f in report_files:
    print(f"  {f}")


103 files in benchmark_reports/:
  benchmark_reports/benchmark_metadata.json
  benchmark_reports/best_per_language.png
  benchmark_reports/ca
  benchmark_reports/ca/cm_m2v-256-distilroberta-base-ca-v2.png
  benchmark_reports/ca/cm_m2v-256-roberta-base-ca-cased-sts.png
  benchmark_reports/ca/cm_m2v-256-roberta-base-ca-v2-massive.png
  benchmark_reports/ca/cm_m2v-256-roberta-large-ca-paraphrase.png
  benchmark_reports/ca/cm_m2v-256-roberta-large-ca-v2-massive.png
  benchmark_reports/ca/cm_m2v-256-roberta-large-ca-v2.png
  benchmark_reports/ca/cm_potion-multilingual-128M.png
  benchmark_reports/ca/f1_m2v-256-distilroberta-base-ca-v2.png
  benchmark_reports/ca/f1_m2v-256-roberta-base-ca-cased-sts.png
  benchmark_reports/ca/f1_m2v-256-roberta-base-ca-v2-massive.png
  benchmark_reports/ca/f1_m2v-256-roberta-large-ca-paraphrase.png
  benchmark_reports/ca/f1_m2v-256-roberta-large-ca-v2-massive.png
  benchmark_reports/ca/f1_m2v-256-roberta-large-ca-v2.png
  benchmark_reports/ca/f1_potion-multi

---
## Summary

This benchmark evaluated **N** model × language combinations across
**N** languages from the OVOS intents dataset.

Check the `benchmark_reports/` directory for:
- **Per-language confusion matrices** and per-class F1 bar charts
- **Model comparison** bar charts per language
- **Cross-language heatmap** of multilingual model performance
- **POTION size-vs-accuracy** scaling curves
- **Multilingual vs language-specific** comparison
- **Full metadata** JSON with all numeric results

> The plots and JSON can be used directly in papers, README files, or
> model cards on HuggingFace.